# DABN13 - Assignment 6

## Preamble: Data
In this lab we are using a dataset on beer purchases. Our goal is to predict if light beer purchased in the US is BUD light. To achieve this goal, we will use the information provided by the following socioeconomic characteristics:
* market           - where the beer is bought
* buyertype        - who is the buyer () 
* income           - ranges of income
* childrenUnder6   - does the buyer have children under 6 years old
* children6to17    - does the buyer have children between 6 and 17
* age              - bracketed age groups
* employment       - fully employed, partially employed, no employment.
* degree           - level of occupation
* occuptation      - which sector you are employed in
* ethnic           - white, asian, hispanic, black or other
* microwave        - own a microwave
* dishwasher       - own a dishwasher
* tvcable          - what type cable tv subscription you have
* singlefamilyhome - are you in a single family home
* npeople          - number of people you live with 1,2,3,4, +5

First, we load the dataset and create an output variable that indicates purchases of Bud Light.

In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import random
np.random.seed(2023)
tf.random.set_seed(2023)
random.seed(2023)


# os.chdir("??") Change working directory if needed 

lb    = pd.read_csv("LightBeer2.csv")
y     = np.zeros(shape=lb.shape[0])
y[lb['beer_brand'] == "BUD LIGHT"]     = 1
demog = lb.iloc[:,9:]
demog = pd.get_dummies(demog, drop_first=True)

# inspect the data
print(lb.shape)
print(demog.shape)
print(y.shape)
# inspect first 5 and last 5
print(lb.head())
print(lb.tail())

(73115, 24)
(73115, 123)
(73115,)
   household   upc_description  quantity beer_brand  beer_spend  beer_floz  \
0    2000946  BUD LT BR CN 12P         1  BUD LIGHT        8.14      144.0   
1    2003036  BUD LT BR CN 24P         1  BUD LIGHT       17.48      288.0   
2    2003036  BUD LT BR CN 24P         2  BUD LIGHT       33.92      576.0   
3    2003036  BUD LT BR CN 30P         2  BUD LIGHT       34.74      720.0   
4    2003036  BUD LT BR CN 36P         2  BUD LIGHT       42.96      864.0   

   price_floz container_descr  promotion          market  ...  age employment  \
0    0.056528             CAN      False  RURAL ILLINOIS  ...  50+       none   
1    0.060694             CAN      False         ATLANTA  ...  50+       full   
2    0.058889             CAN      False         ATLANTA  ...  50+       full   
3    0.048250             CAN      False         ATLANTA  ...  50+       full   
4    0.049722             CAN      False         ATLANTA  ...  50+       full   

    degree

We also split the data into training and test sets:

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(demog, y, train_size=0.75, shuffle=False)

stdz_X = StandardScaler().fit(X_train)

X_train = stdz_X.transform(X_train)
X_test  = stdz_X.transform(X_test)

## Part 1: Specifying and training neural networks
We will now start building a neural network to predict the purchase of Bud Light.

### Task 1a) 
We start with specifying the architecture of our very first and very small neural net `model1`.
Add three layers to `model1`, two hidden layers with $30$ and $15$ hidden units, respectively, and an output layer.
For the two hidden layers you should use the ReLU activation function. Additionally, choose a suitable activation for the output layer, given that we have a classification problem. See [the Keras documentation](https://keras.io/api/layers/activations/) for activation functions. 

In [3]:
from tensorflow import keras
from tensorflow.keras import layers

# Initialize a first model
model1 = keras.Sequential()

# Add layers to the model
model1.add(layers.Dense(30, input_dim=X_train.shape[1], activation='ReLU'))
model1.add(layers.Dense(15, activation = 'ReLU'))
model1.add(layers.Dense(1, activation='sigmoid'))

2023-10-14 16:19:08.308844: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2023-10-14 16:19:08.308868: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-10-14 16:19:08.308876: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-10-14 16:19:08.308962: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-14 16:19:08.308980: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


### Task 1b) 

Next, we compile our model specification. From [https://keras.io/api/losses/probabilistic_losses/](losses) select a suitable loss function for our classification problem. As optimization algorithm use *Adam* with learning rate $0.00003$. Lastly, use `accuracy` as a metric.

In [4]:
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.losses import binary_crossentropy

# Compile the model
model1.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.00003), metrics=['accuracy'])

### Task 1c)
Now train the model using $250$ epochs, a batch_size of $2^8$, and use $25\%$ of the data for validation.
Use the string variable `loss_valloss_difference_1c` to describe and explain the observed difference between validation loss and training loss.

In [5]:
model1.fit(X_train, y_train, epochs=250, batch_size=2**8, validation_split=0.25)

loss_valloss_difference_1c = "Training loss indicates how well the model is fitting the training data, whereas the validation loss indicates how well the model is generalizing to new data. At some point the validation loss actually increases, which indicates that the model is overfitting the training data. Ideally we would want to use less epochs so that the model does not overfit the training data, because this in turn also affects the accuracy of the model.  "

Epoch 1/250


2023-10-14 16:19:13.350190: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-10-14 16:19:13.630659: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


161/161 [==============================] - ETA: 0s - loss: 0.9432 - accuracy: 0.5110

2023-10-14 16:19:21.320371: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


161/161 [==============================] - 10s 31ms/step - loss: 0.9432 - accuracy: 0.5110 - val_loss: 0.8546 - val_accuracy: 0.5427
Epoch 2/250
161/161 [==============================] - 2s 9ms/step - loss: 0.8918 - accuracy: 0.5210 - val_loss: 0.8140 - val_accuracy: 0.5587
Epoch 3/250
161/161 [==============================] - 1s 9ms/step - loss: 0.8508 - accuracy: 0.5238 - val_loss: 0.7804 - val_accuracy: 0.5707
Epoch 4/250
161/161 [==============================] - 1s 9ms/step - loss: 0.8177 - accuracy: 0.5321 - val_loss: 0.7516 - val_accuracy: 0.5837
Epoch 5/250
161/161 [==============================] - 1s 9ms/step - loss: 0.7907 - accuracy: 0.5412 - val_loss: 0.7269 - val_accuracy: 0.6016
Epoch 6/250
161/161 [==============================] - 1s 9ms/step - loss: 0.7686 - accuracy: 0.5459 - val_loss: 0.7055 - val_accuracy: 0.6020
Epoch 7/250
161/161 [==============================] - 1s 9ms/step - loss: 0.7503 - accuracy: 0.5522 - val_loss: 0.6868 - val_accuracy: 0.6159
Epoch 8/2

### Task 1d)
In Lecture 9 we used early stopping to avoid overfitting. Apply this here, with `patience` argument set to 20, a new model `model1b` which otherwise should have a setup identical to `model1`. In which epoch did the model training procedure stop? Figure this out by counting the number of elements in `model1b_fit.history['loss']` and write your answer into the string variable `when_earlystop_1d`.

In [7]:
from tensorflow.keras.callbacks import EarlyStopping

# Define architecture here
model1b = keras.Sequential()
model1b.add(layers.Dense(30, input_dim=X_train.shape[1], activation='ReLU'))
model1b.add(layers.Dense(15, activation = 'ReLU'))
model1b.add(layers.Dense(1, activation='sigmoid'))

# Compile model here 
model1b.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.00003), metrics=['accuracy'])

# Fit model here
model1b_fit = model1b.fit(X_train, y_train, epochs=250, batch_size=2**8, validation_split=0.25, callbacks=[EarlyStopping(patience=20)])

when_earlystop_1d = f"Early stopping is used to stop the training of the model when a monitored\nmetric has stopped improving. In this case the monitored\nmetric is the validation loss. The patience parameter\nis used to specify how many epochs the model should\nwait before stopping the training. In this case\nthe model stopped training after {len(model1b_fit.epoch)} epochs."

print(when_earlystop_1d)

Epoch 1/250
161/161 [==============================] - 0s 1ms/step - loss: 0.7777 - accuracy: 0.4800 - val_loss: 0.8446 - val_accuracy: 0.3018
Epoch 2/250
161/161 [==============================] - 0s 1ms/step - loss: 0.7399 - accuracy: 0.4967 - val_loss: 0.7632 - val_accuracy: 0.4357
Epoch 3/250
161/161 [==============================] - 0s 899us/step - loss: 0.7144 - accuracy: 0.5259 - val_loss: 0.7054 - val_accuracy: 0.5207
Epoch 4/250
161/161 [==============================] - 0s 902us/step - loss: 0.6968 - accuracy: 0.5480 - val_loss: 0.6647 - val_accuracy: 0.5753
Epoch 5/250
161/161 [==============================] - 0s 907us/step - loss: 0.6837 - accuracy: 0.5724 - val_loss: 0.6366 - val_accuracy: 0.6321
Epoch 6/250
161/161 [==============================] - 0s 899us/step - loss: 0.6733 - accuracy: 0.5979 - val_loss: 0.6164 - val_accuracy: 0.6610
Epoch 7/250
161/161 [==============================] - 0s 899us/step - loss: 0.6647 - accuracy: 0.6058 - val_loss: 0.6021 - val_accura

### Task 1e)
Even though we haven't finished training our neural net, let us use the `evaluate()` function to measure the predictive performance of `model1b` on the test data. Save the result as `res_model1`. 

What is the accuracy of the model for validation training data and test data, respectively? What is the difference in accuracy? Save your answer in the string variable `difference_in_accuracy_1e`.
*Hint:* the training validation accuracy can be extracted from `model1b_fit`.


In [8]:
res_model1b = model1b.evaluate(X_test, y_test, verbose = 0)

print(res_model1b)

difference_in_accuracy_1e = f"The accuracy for the validation training data is\n{model1b_fit.history['val_accuracy'][-1]} and the test data accuracy is {res_model1b[1]} and he difference between\nthe model for validation training data accuracy and test data accuracy is\n{abs(model1b_fit.history['val_accuracy'][-1]) - res_model1b[1]}"

print(difference_in_accuracy_1e)

# print the accuracy of the model for validation training data and test data
#print(f"Accuracy for validation training data: {model1b_fit.history['accuracy'][-1]}")
#print(f"Accuracy for test data: {res_model1b[1]}")


[0.5708325505256653, 0.7084632515907288]
The accuracy for the validation training data is
0.7100445032119751 and the test data accuracy is 0.7084632515907288 and he difference between
the model for validation training data accuracy and test data accuracy is
0.0015812516212463379


### Task 1f)
Now we use the `confusion_matrix()` function from the `metrics` module of scikit-learn to disaggregate model performance to class-specific performance. First, get class predictions on the test data using `predict()`. Save these as `prob_model1`. 
Second, use `confusion_matrix()` to get a confusion matrix and save it as `CM_model1`. Third, calculate true positive rate and false positive rate and save them as `TPR_1f` and `FPR_1f`. 
Do your results on TPR and FPR suggest that prediction accuracy is approximately equal in both categories? Write your (specific!) answer into the string variable `categorywise_accuracy_1f`

In [9]:
from sklearn.metrics import confusion_matrix

prob_model1 = model1b.predict(X_test)
CM_model1   = confusion_matrix(y_test, prob_model1 > 0.5)

TN = CM_model1[0,0]
TP = CM_model1[1,1]
FN = CM_model1[1,0]
FP = CM_model1[0,1]


TPR_1f = TP/(TP + FN)
FPR_1f = FP/(FP + TN)

print(CM_model1)
print(TPR_1f)
print(FPR_1f)

categorywise_accuracy_1f = "The model is much better at predicting true negatives, where approximately 80 percent of true negatives are predicted correctly, whereas true positive is only approximately 47 percent correctly predicted. The model is also better at predicting false negatives, where approximately 53 percent of false negatives are predicted correctly, whereas false positives is only approximately 20 percent correctly predicted. One could consider decreasing the decision boundary to increase the true positive rate, but this would in turn increase the false positive rate. Which would be worse than the current situation, because the model is better at predicting true negatives than false positives. "


572/572 [==============================] - 0s 330us/step
[[10416  2465]
 [ 2864  2534]]
0.46943312337902926
0.1913671298812204


### Task 1g)

In the lectures we have utilized explicit regularization to avoid overfitting. Here we will use $\ell_2$ regularization to update the weights. Create the architecture of a new neural net `model2` which is identical to that of `model1b` except for $l_2$ regularization with regularization factor `l2_pen` in the two hidden layers.  

Then compile and fit this regularized model with the same parameters as in Task 1d. Save the trained neural net as `model2_fit`.

In [10]:
from tensorflow.keras.regularizers import l2
l2_pen = 0.005

# 1.
model2 = keras.Sequential()
model2.add(layers.Dense(30, input_dim = X_train.shape[1], activation = 'ReLU', kernel_regularizer = l2(l2_pen)))
model2.add(layers.Dense(15, activation = 'ReLU', kernel_regularizer = l2(l2_pen)))
model2.add(layers.Dense(1, activation = 'sigmoid'))

# 2.
model2.compile(loss = 'binary_crossentropy', optimizer = Adam(learning_rate = 0.00003), metrics = ['accuracy'])
model2.fit(X_train, y_train, epochs = 250, batch_size = 2**8, validation_split = 0.25, callbacks=[EarlyStopping(patience=20)])

Epoch 1/250
161/161 [==============================] - 0s 1ms/step - loss: 1.0621 - accuracy: 0.5544 - val_loss: 0.8882 - val_accuracy: 0.7022
Epoch 2/250
161/161 [==============================] - 0s 948us/step - loss: 1.0345 - accuracy: 0.5581 - val_loss: 0.8819 - val_accuracy: 0.7027
Epoch 3/250
161/161 [==============================] - 0s 947us/step - loss: 1.0111 - accuracy: 0.5732 - val_loss: 0.8729 - val_accuracy: 0.7101
Epoch 4/250
161/161 [==============================] - 0s 903us/step - loss: 0.9909 - accuracy: 0.5831 - val_loss: 0.8629 - val_accuracy: 0.7189
Epoch 5/250
161/161 [==============================] - 0s 908us/step - loss: 0.9730 - accuracy: 0.5936 - val_loss: 0.8523 - val_accuracy: 0.7371
Epoch 6/250
161/161 [==============================] - 0s 920us/step - loss: 0.9568 - accuracy: 0.6056 - val_loss: 0.8428 - val_accuracy: 0.7432
Epoch 7/250
161/161 [==============================] - 0s 999us/step - loss: 0.9421 - accuracy: 0.6157 - val_loss: 0.8337 - val_accu

### Task 1h)
In Task 1e) we compared the prediction accuracy on test and training sets. However, this is bad measure when the data is not well balanced (in terms of the observed output categories). Instead, one can use the cross entropy for the binomial distribution (minus the average log likelihood of the model). In fact, we chose this function as loss function for model training when we compiled `model1` and `model2`.

To compare the test error of `model2` to that of `model1b` we don't want to use `loss` from `evaluate` since this includes the $\ell_2$ penalty.  In the library `MLmetrics` the function `log_loss()` computes the cross entropy for the binomial distribution without penalty term. 

First, get predicted output probabilities on test data from `model2` and save them as `prob_model2`.
Second, use `log_loss()` from the metrics module in scikit-learn to compute the cross-entropy loss for `model2` on the test data and save it as `logloss_model2`. 
Third, use the string variable `performance_comparison_1h` to describe how the accuracy on test data differs between `model1b` and `model2`.

In [11]:
from sklearn.metrics import log_loss

# 1. 
prob_model2    = model2.predict(X_test)

# 2.
logloss_model2 = log_loss(y_test, prob_model2)
print(logloss_model2)

# 3.
accu_model2     = model2.evaluate(X_test, y_test, verbose = 0)[1]
performance_comparison_1h = f"From the log loss we can that the model is performing even better when adding a penalty,\nquite significantly actually, the log loss is {logloss_model2:.6f} compared to {res_model1b[0]:.6f}\nfor model 1b. The accuracy achieved is {accu_model2:.6f} with\na penalty and {res_model1b[1]:.6f} without a penalty. The difference between loss of the two models\nis {abs(logloss_model2 - res_model1b[0]):.6f} and the difference between the accuracy of the two models is\n{abs(accu_model2 - res_model1b[1]):.6f}. One could also imagine that if even more features were added to the model,\nthe penalty term would perform even better."
print(performance_comparison_1h)


572/572 [==============================] - 0s 360us/step
0.5360111679102018
From the log loss we can that the model is performing even better when adding a penalty,
quite significantly actually, the log loss is 0.536011 compared to 0.570833
for model 1b. The accuracy achieved is 0.736857 with
a penalty and 0.708463 without a penalty. The difference between loss of the two models
is 0.034821 and the difference between the accuracy of the two models is
0.028393. One could also imagine that if even more features were added to the model,
the penalty term would perform even better.


## Part 2: Tuning neural nets with caret

Keras provides functions that allow the use of scikit-learn for model tuning. Using this functionality requires relatively little effort and in this part we are going to practice the individual steps of model tuning with `sklearn`.


### Task 2a)
First, we need to define the architecture of the neural net that we tune and to compile the model. This needs to be done inside a function. The arguments of this function are the tuning parameters whose candidate values we want to feed into the function one by one.

In this task, we work with the architecture defined for `model2` in Task 1g. The only parameter that we want to tune is the regularization parameter for $\ell_2$ penalization inside the hidden units.

Now create a function `modelbuild_2a` which has one argument `l2_pen`. In this function, specify the architecture of a Keras model `model`, identical to that of `model2` and with $\ell_2$-penalty set to `ell_2`. Compile the model with the same settings as in Task 1g and return it as function output.

In [12]:
def modelbuild_2a(l2_pen=0.0):
    ell_2 = l2_pen
    model_2a = keras.Sequential()
    model_2a.add(layers.Dense(30, input_dim = X_train.shape[1], activation = "ReLU", kernel_regularizer=l2(ell_2)))
    model_2a.add(layers.Dense(15, activation = "ReLU", kernel_regularizer=l2(ell_2)))
    model_2a.add(layers.Dense(1, activation= "sigmoid"))
    model_2a.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.00003), metrics=["accuracy"])
    return(model_2a)

### Task 2b)
In order to make the function `modelbuild_2a` compatible with `sklearn`, we need to call it inside the `KerasClassifier()` function from the `wrappers` module of `scikit-learn`. As additional arguments, provide all arguments that you used when fitting `model2`.

In [14]:
from scikeras.wrappers import KerasClassifier


model2_sklearn_spec = KerasClassifier(build_fn=modelbuild_2a, l2_pen=0.0,
                                      epochs=250, 
                                      batch_size=2**8, 
                                      verbose=0, 
                                      validation_split=0.25, 
                                      callbacks=[EarlyStopping(patience=20)])

### Task 2c)
Next, define a parameter grid `tune_grid_2c`. This must be a dictionary object. Ensure that the only object within `tune_grid_2c` is called `l2_pen`. Its values should be zero as well as $10^{r}$ for a grid of eleven $r$-values from $-4$ and $-1$ at equal distance.

In [15]:
tune_grid_2c = {
"l2_pen": np.insert(np.logspace(-4, -1, 11), 0, 0).tolist()
}

### Task 2d)
Now, we can tune our model. That's computationally quite costly, so we will use merely a fraction of the available training data. The inputs and outputs for this task are given by

In [16]:
X_train_small,_ , y_train_small, _ = train_test_split(X_train, y_train, train_size=0.3, random_state=6)

Do the following:

1. Use `Kfold()` from the model selection module in scikit-learn to define a random partition of the training data into five folds. Use $5$ as your random seed. Save this partition as `cv_splits_2d`
2. Call `GridSearchCV()` and use the wrapper function from Task 2b, the parameter grid from Task 2c as well as `cv_splits_2d` as arguments
3. Apply the `fit()`-method to `GridSearchCV` and use `X_train_small` and `y_train_small` as inputs and outputs, respectively.

In [17]:
from sklearn.model_selection import (KFold, GridSearchCV)
import time

start_time = time.time()

# 1.
cv_splits_2d = KFold(n_splits=5, shuffle=True, random_state=5)

# 2.
NN_tune_2d = GridSearchCV(estimator=model2_sklearn_spec, 
                          param_grid=tune_grid_2c,
                          cv=cv_splits_2d)

# 3.
NN_tune_2d.fit(X_train_small, y_train_small)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Cross validation finished in {elapsed_time} seconds")

/Users/asger/anaconda3/envs/my_new_env/lib/python3.9/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/asger/anaconda3/envs/my_new_env/lib/python3.9/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/asger/anaconda3/envs/my_new_env/lib/python3.9/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/asger/anaconda3/envs/my_new_env/lib/python3.9/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` w

Cross validation finished in 668.5910530090332 seconds


### Task 2e)
By default, GridSearchCV saves the trained model with the best tuning parameter values as as object `best_estimator_` inside `NN_tune_2d`. However, in our case this model is a KerasClassifier object. We cannot use such an object for making predictions on test data. 

Still, the KerasClassifier object contains the trained model in the typical Keras format as object `model`. Extract this object-inside-the-object-inside-the-object and save it as `model3`.

In [7]:
model3 = NN_tune_2d.best_estimator_.model_
print(model3.summary())

NameError: name 'NN_tune_2d' is not defined

## Part 3: Saving, loading and retraining neural nets 

### Task 3a)

Training and tuning neural nets can take a lot of time. Therefore, it is possible to save entire fitted models to disk and to import them at a later point in time. Apply the `save()`-method to your most recent `model3` in order to save it as *DABN13_asst6_saved_model3* in your working directory.

Ensure that the model is saved in TensorFlow SavedModel format.

Additionally, use the file explorer in your operating system to look how exactly the model was saved on your hard drive. Describe this shortly in the string_variable `saved_model_3a`

*Note:* Functions for saving and loading models are very nicely described in the [keras documentation](https://keras.io/api/saving/model_saving_and_loading/#save-method)




In [6]:
model3.save()

saved_model_3a = "??"

NameError: name 'model3' is not defined

### Task 3b)

Now, use the `load_model` function in the `models` module of Keras to load your saved model into your python session again. Save this model as `model4`.

*Note:* The possibility to load a previously saved model from your hard disk is useful for more than just your own models. It even allows you to load pretrained models for specific purposes from the TensorFlow Hub or from Hugging Face. These models could then directly be used for prediction or fine-tuned on your data.

In [ ]:
from tensorflow.keras.models import load_model

model4 = ??


### Task 3c)

When we tuned our most recent neural net, we did this on a relatively small fraction of the training data to reduce the computational cost. This was also the data used to train the best model that we extracted from `NN_tune_2d`.
Now that we have chosen an optimal tuning parameter, it makes sense to retrain the `model4` on the entire training data `X_train` and `y_train`. Do this by applying the `fit()` method to `model4`. As previously, training should be done for 250 epochs, unless early stopping with a patience of 20 epochs kicks in. Minibatches of $2^8$ data points should be used. Given the large amount of data, hold only 10% of the data aside for monitoring validation loss.

Once you retrained your model, obtain predicted class probabilities and save them as `prob_model4`. Then, get the log loss `logloss_model4` on the test data.

Finally, to what extend did model tuning and retraining change test set accuracy relative to that of `model2`? Comment on this in the string variable `performance_comparison_3c`

In [ ]:
# 1.
??

# 2.
prob_model4    = ??
logloss_model4 = ??
print(logloss_model4)

# 3.
performance_comparison_3c = "??"

## Part 4: Manual predictions from a trained neural net

In this part we will build predictions *manually* by extracting weights from the trained  `model2` and by constructing the transformations in the layers of the neural net ourselves.

### Task 4a)

Start this task by creating your own ReLU activation function. Save it as `ReLU`. Then, write your own sigmoid function for the output layer transformation. Save it as  `sigmoid`. 

In [ ]:
# ReLU activation function
??

# Sigmoid activation function
??

### Task 4b)
In the slides for lecture 8 we discussed how units in the different layers of a neural net look like. Now we are going to use the equations both hidden units and output unit to construct output predictions for the $n_{test}$ data points in `X_test`. Please do the following:

1. Apply the `get_weights()` method on `model2` to obtain a list object which stores the weights and biases of the learned model. Save it as `weight_and_bias_4b`.
2. Extract the objects inside `weight_and_bias_4b` into the objects for $\mathbf{b}_1,\mathbf{W}_1, \mathbf{b}_2, \mathbf{W}_2,\mathbf{b}_3, \mathbf{W}_3$ defined in the code chunk below.
3. Construct the linear term of the first layer hidden units and save these linear terms as a $30 \times n_{train}$ vector `Z_1`.
4. Construct the $15 \times n_{train}$ vector `Z_2` of linear terms for the second layer hidden units. Then, obtain the linear term of the output unit and save it as `Z_3`.
5. Put `Z_3` into the output layer activation function in order to get predictions for the probability of a Bud Light purchase. Save this as $n_{train} \times 1$ vector `pred_own_2b`. 

*Hint*: You can use `dim()` to check the dimension of matrices and `length()` to check that of vectors. Additionally, to ensure that you got the correct result for `prob_model2_own_4b` you can compare it with the output of `predict()`.

In [ ]:
# 1.
weight_and_bias_4b = ??

# 2.
WW_1 = ??
bb_1 = ??
WW_2 = ??
bb_2 = ??
WW_3 = ??
bb_3 = ??

# 3.
Z_1 = ??

# 4.
Z_2 = ??
Z_3 = ??

# Apply sigmoid activation to Z_3
prob_model2_own_4b = ??